## 사전작업

In [1]:
# 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [15]:
import pandas as pd
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import matplotlib.font_manager as fm
# 설치된 폰트 중 'NanumGothic' 찾기
for font in fm.fontManager.ttflist:
    if 'NanumGothic' in font.name:
        plt.rcParams['font.family'] = font.name
        break
plt.rcParams['axes.unicode_minus'] = False
import seaborn as sns
import numpy as np
import scipy.stats as stats

## 데이터 로드

### 데이터 로드 및 원하는 데이터 추출

In [30]:
df_elderly = pd.read_csv('./elderly.csv')
df_elderly

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
1,66445cd65350d17a77350dc6,1269,밝은내데이케어센터 전문강사 채용 공고,밝은내데이케어센터,24-03-11,187,2024년 밝은내데이케어센터 전문강사 채용 공고\n 밝은내데이케어센터는 이용 어르...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
3,66445cfd5350d17a77350dc8,1267,참사람들시니어케어 사회복지사 채용(긴급) 1명,성남재가노인복지센터,24-02-21,187,Ⅰ. 모집분야 및 자격기준\n1. 채용인원 : 장기요양서비스(방문요양) 사회복지사 ...
4,66445d035350d17a77350dc9,1266,[채용]시립마포노인종합복지관병설 데이케어센터 사회복지사 채…,마포데이케어센터,24-01-05,320,[채용] 마포데이케어센터 공고 제 2024-01호\n 마포데이케어센터 사회복지사 ...
...,...,...,...,...,...,...,...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...
1242,664478245350d17a7735129f,43,단원구노인복지관 방문요양 요양보호사 모집,NaN,08-09-19,13202,▒ 기관정보 ▒\n------------------------------------...


In [31]:
df_elderly_2 = df_elderly[df_elderly['title'].str.contains('간호조무사|요양보호사', na=False)]
df_elderly_2

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,23-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1235,664477fe5350d17a77351298,50,요양보호사 및 사회복지사 채용 공고,NaN,08-10-02,14300,▒ 기관정보 ▒\n------------------------------------...
1236,664478035350d17a77351299,49,경기 안성 감로당 가정봉사원파견센터 요양보호사 1급 모집,감로당단기보호,08-10-02,14299,첨부파일을 참조해 주세요~
1237,664478085350d17a7735129a,48,대구 요양보호사1급 모십니다,NaN,08-09-30,15109,▒ 기관정보 ▒\n------------------------------------...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...


In [32]:
df_elderly_3 = df_elderly[df_elderly['content[0]'].str.contains('간호조무사|요양보호사', na=False)]
df_elderly_3

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
8,66445d195350d17a77350dcd,1262,[채용]시립마포노인종합복지관 데이케어센터 파트타임 간호조무…,마포데이케어센터,23-05-10,1786,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
9,66445d1f5350d17a77350dce,1261,[긴급채용]시립마포노인종합복지관 데이케어센터 파트타임 요양…,마포데이케어센터,23-05-10,1946,[채용] 마포데이케어센터 공고 제 2023-02호\n 마포데이케어센터 요양보호사 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
...,...,...,...,...,...,...,...
1239,664478135350d17a7735129c,46,요양보호사(사회적일자리) 모십니다,NaN,08-09-26,14116,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...
1242,664478245350d17a7735129f,43,단원구노인복지관 방문요양 요양보호사 모집,NaN,08-09-19,13202,▒ 기관정보 ▒\n------------------------------------...


In [33]:
df_concat = pd.concat([df_elderly_2,df_elderly_3])
df_concat = df_concat.drop_duplicates()
df_concat

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,24-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,24-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,23-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,23-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,23-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1213,664477845350d17a77351282,72,도솔노인복지센터 직원채용공고,NaN,08-10-20,14389,▒ 기관정보 ▒\n------------------------------------...
1238,6644780e5350d17a7735129b,47,직원채용공고,NaN,08-09-28,14125,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,08-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,08-09-22,14426,▒ 기관정보 ▒\n------------------------------------...


### 모집한 데이터를 분석

In [34]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   _id         771 non-null    object
 1   number      771 non-null    int64 
 2   title       771 non-null    object
 3   writter     684 non-null    object
 4   date        771 non-null    object
 5   count       771 non-null    int64 
 6   content[0]  771 non-null    object
dtypes: int64(2), object(5)
memory usage: 48.2+ KB


In [35]:
df_concat['date'] = pd.to_datetime(df_concat['date'], format='%y-%m-%d')
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   _id         771 non-null    object        
 1   number      771 non-null    int64         
 2   title       771 non-null    object        
 3   writter     684 non-null    object        
 4   date        771 non-null    datetime64[ns]
 5   count       771 non-null    int64         
 6   content[0]  771 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 48.2+ KB


In [36]:
df_concat

,_id,number,title,writter,date,count,content[0]
0,66445cbb5350d17a77350dc5,1270,은평데이케어센터 주간 요양보호사 구인,시립은평노인종합복지관 병설 …,2024-05-07,31,은평데이케어센터에서는 가족 같은 마음으로 어르신들을 돌봐주실 요양보호사를 아래와\n...
2,66445cf85350d17a77350dc7,1268,도계재가노인복지센터 요양보호사 채용공고,도계재가노인복지센터,2024-03-07,172,
7,66445d135350d17a77350dcc,1263,구립 역삼데이케어센터 직원(요양보호사) 채용 공고,구립역삼데이케어센터,2023-11-03,521,구립 역삼데이케어센터 직원(요양보호사) 채용 공고\n강남구립 역삼데이케어센터에서는 ...
12,66445d2f5350d17a77350dd1,1258,[채용] 시립마포노인종합복지관별설 데이케어센터 요양보호사 …,마포데이케어센터,2023-04-13,2180,"「마포데이케어센터」에서는 센터의 미션을 함께 이루고, 어르신을 가족처럼 따듯하게 섬..."
13,66445d345350d17a77350dd2,1257,◆번동데이케어센터 야간요양보호사 채용공고◆,번동데이케어센터,2023-04-07,1366,1. 모집분야\n가. 채용내용\n 채용 분야\n인원\n근무조건\n담당업무\n근무기간...
...,...,...,...,...,...,...,...
1213,664477845350d17a77351282,72,도솔노인복지센터 직원채용공고,NaN,2008-10-20,14389,▒ 기관정보 ▒\n------------------------------------...
1238,6644780e5350d17a7735129b,47,직원채용공고,NaN,2008-09-28,14125,▒ 기관정보 ▒\n------------------------------------...
1240,664478195350d17a7735129d,45,새로 open하는 노인복지센터입니다,NaN,2008-09-26,14318,▒ 기관정보 ▒\n------------------------------------...
1241,6644781f5350d17a7735129e,44,[직원채용공고] 보령주간노인복지센터 직원채용공고,NaN,2008-09-22,14426,▒ 기관정보 ▒\n------------------------------------...


In [37]:
# 'date' 열에서 연도를 추출하여 'year' 열을 생성
df_concat['year'] = df_concat['date'].dt.year

# 'year' 열을 기준으로 그룹화하고 각 그룹의 크기를 계산
year_counts = df_concat.groupby('year').size()

print(year_counts)


year
2008     71
2009    153
2010     90
2011     77
2012     64
2013     53
2014     45
2015     26
2016     29
2017     33
2018     26
2019     15
2020     20
2021     32
2022     15
2023     18
2024      4
dtype: int64


In [38]:
df_elderly['date'] = pd.to_datetime(df_elderly['date'], format='%y-%m-%d')
df_elderly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244 entries, 0 to 1243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   _id         1244 non-null   object        
 1   number      1244 non-null   int64         
 2   title       1244 non-null   object        
 3   writter     1122 non-null   object        
 4   date        1244 non-null   datetime64[ns]
 5   count       1244 non-null   int64         
 6   content[0]  1242 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 68.2+ KB


In [39]:
# 'date' 열에서 연도를 추출하여 'year' 열을 생성
df_elderly['year'] = df_elderly['date'].dt.year

# 'year' 열을 기준으로 그룹화하고 각 그룹의 크기를 계산
year_count = df_elderly.groupby('year').size()

print(year_count)

year
2008    100
2009    220
2010    127
2011    121
2012     95
2013     82
2014     71
2015     48
2016     50
2017     65
2018     60
2019     48
2020     41
2021     52
2022     27
2023     25
2024     12
dtype: int64
